In [44]:
# Load packages
import pandas as pd
import numpy as np
import lxml.html as lx
import requests
import requests_cache
from newspaper import Article

requests_cache.install_cache("mycache")

In [3]:
# No need to use this.
# Read the api key. I got this function from Nick's lecture notes from 1/31.
def read_key(keyfile):
    with open(keyfile) as f:
        return f.readline().strip("\n")
    
key = read_key("../keys/googlenews.txt")

In [58]:
# No need to use this.
response = requests.get("https://newsapi.org/v2/everything", params = {
    "q": "camp fire",
    "sortBy": "relevancy",
    "apiKey": key
})

response.raise_for_status()

r = response.json()

{'status': 'ok',
 'totalResults': 2306,
 'articles': [{'source': {'id': 'the-new-york-times',
    'name': 'The New York Times'},
   'author': 'JILL COWAN',
   'title': 'California Today: California Today: Why ‘the Worst of Both Worlds’ Is in the Forecast',
   'description': 'Friday: Californians will most likely face more extreme weather; Pacific Gas & Electric’s role in the Camp Fire; San Diego treats',
   'url': 'https://www.nytimes.com/2019/03/01/us/california-today-extreme-weather-forecast.html',
   'urlToImage': 'https://static01.nyt.com/images/2019/03/01/us/01campfirecatoday/merlin_151063710_b332153e-e314-410c-8eef-5c41339f26bd-facebookJumbo.jpg',
   'publishedAt': '2019-03-01T14:09:58Z',
   'content': 'Im heading to San Diego this weekend, which feels like a real stroke of luck, in light of this weeks recommendation from Tejal Rao, The Timess California restaurant critic. But even if youre far away, take this as an invitation to treat yourself to your favor… [+1197 chars]'},
  {

In [95]:
# Test the newspaper function.
link = "https://www.nytimes.com/2019/02/28/business/energy-environment/pge-camp-fire.html"
#link = 'https://krcrtv.com/news/butte-county/highway-70-to-be-closed-for-emergency-camp-fire-repairs'
article = Article(link)
article.download()
article.parse()
article.text

'Pacific Gas & Electric said Thursday that its equipment had probably caused the Camp Fire, the catastrophic November blaze that destroyed thousands of homes in Paradise, Calif., and killed at least 86 people.\n\nPG&E, which filed for bankruptcy protection in January, said it had recorded a $10.5 billion charge in anticipation of damage claims for that fire, the deadliest in state history. Largely as a result, the company reported a $6.9 billion loss for 2018.\n\nThough the cause of the fire is still under official investigation by California officials, PG&E said it “believes it is probable that its equipment will be determined to be an ignition point of the 2018 Camp Fire.” Attempts to determine the fire’s cause center on the 56-mile Caribou-Palermo electric transmission line.\n\nThe California Department of Forestry and Fire Protection has said the fire started at 6:33 a.m. on Nov. 8 near a tower on that line. PG&E said that its line de-energized at 6:15 a.m., adding that 15 minutes 

In [94]:
#article.summary
#article.keywords
article.publish_date

datetime.datetime(2019, 2, 28, 0, 0)

In [59]:
# Get google news links for each search term
search_term_links = [
    "https://news.google.com/search?q=camp%20fire&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20death%20toll&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20california&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20paradise&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20map&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20news&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20cleanup&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20debris%20removal&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20pg&e&hl=en-US&gl=US&ceid=US%3Aen",
    "https://news.google.com/search?for=camp%20fire%20investigation&hl=en-US&gl=US&ceid=US%3Aen"
]

In [100]:
# Get article links
def get_links(url):
    """Gets the article links for one Chronicle article list. The parameter "url" is the url to the 
    article list."""
    
    response = requests.get(url)
    response.raise_for_status()
    html = lx.fromstring(response.text)
    html.make_links_absolute(url)
    
    # Get the links that contain the word "article".
    #links = html.xpath("//div/a/@href")
    links = html.xpath("//*[contains(@class, 'NiLAwe y6IFtc R7GTQ keNKEd j7vNaf nID9nc')]/a/@href")

    return links

In [103]:
# Get article links for each search term
l = [get_links(l) for l in search_term_links]

# Unlist the links and get the unique links.
links = [item for sublist in l for item in sublist]
links = np.unique(links)

703

In [51]:
links = get_links("https://news.google.com/topics/CAAqKAgKIiJDQkFTRXdvTkwyY3ZNVEZtYUc1bWVuRnFaaElDWlc0b0FBUAE?hl=en-US&gl=US&ceid=US%3Aen")
links[0]

'https://www.cnn.com/2019/02/08/us/camp-fire-deaths/index.html'

In [106]:
# Extract text and publish data for the urls

def get_article_info(link):
    """Extract the title, text, publish date from a news article. The parameter link is the article link.
    Returns a dictionary of title, text, publish date, and url."""
    
    article = Article(link)
    article.download()
    article.parse()
    
    title = article.title
    text = article.text
    date = article.publish_date
    url = link
    
    res = {"url": url, 
           "title": title, 
           "text": text,
           "date": date}
    return res

In [107]:
# test it
get_article_info(link)

{'url': 'https://www.nytimes.com/2019/02/28/business/energy-environment/pge-camp-fire.html',
 'title': 'PG&E Says It Probably Caused the Fire That Destroyed Paradise, Calif.',
 'text': 'Pacific Gas & Electric said Thursday that its equipment had probably caused the Camp Fire, the catastrophic November blaze that destroyed thousands of homes in Paradise, Calif., and killed at least 86 people.\n\nPG&E, which filed for bankruptcy protection in January, said it had recorded a $10.5 billion charge in anticipation of damage claims for that fire, the deadliest in state history. Largely as a result, the company reported a $6.9 billion loss for 2018.\n\nThough the cause of the fire is still under official investigation by California officials, PG&E said it “believes it is probable that its equipment will be determined to be an ignition point of the 2018 Camp Fire.” Attempts to determine the fire’s cause center on the 56-mile Caribou-Palermo electric transmission line.\n\nThe California Departme